In [2]:
from urllib import request
import datetime
import requests
from bs4 import BeautifulSoup
import pytz
from google.cloud import bigquery
import os
import sys
from dotenv import load_dotenv
import pandas as pd
import json

# Load env variables from .env
load_dotenv()

# Grab AIRFLOW_HOME env variable
AIRFLOW_HOME = os.getenv('AIRFLOW_HOME')

# Add airflow to sys.path
sys.path.append(AIRFLOW_HOME)

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = f'{AIRFLOW_HOME}/config/ServiceKey_GoogleCloud.json'

latest_link = 'pageviews-20230621-130000.gz'

latest_link = latest_link.replace("pageviews-", "")[:-3]

latest_link

'20230621-130000'

In [ ]:
In the following series of articles I will implement a data product for predicting the price fluctuation of major companies in the stock market by analyzing Wikipedia pageviews.

A pageview is a request to load a single page of an internet website. On Wikipedia, a pageview is defined as a request for the content of any page served from the Wikimedia server. 
The pageview data is publicly available at https://dumps.wikimedia.org/other/pageviews/.

The main assumption is that the pageviews of a company on Wikipedia are correlated with the price fluctuation of the company in the stock market. The more people view the company's page on Wikipedia, the more likely the company's stock price will go up. The less people view the company's page on Wikipedia, the more likely they are interested the company and willing to buy the company's stock, which will drive the company's stock price up.

The data product will be implemented using the following tools:

Apache Airflow for data pipeline management
Python for data ingestion and processing
Google BigQuery for data storage and processing
Docker for containerization

The first article will focus on setting up a data pipeline that will automatically load pageviews of several major companies from the last hour into BigQuery.
It will also load the live stock prices of the companies from Yahoo Finance into BigQuery.

Every 15 minutes the pipeline will check if a new file of pageviews has arrived to Wikipedia servers, and if so,  it will load both the pageviews and the stock prices into BigQuery.

The second article will focus on building a machine learning model that will predict the price fluctuation of the companies based on the pageviews and stock prices.







In [1]:
import pandas as pd
import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

df = pd.read_csv("/projects/predicting_stocks_using_pageviews/test.csv", sep='\t')

df['year'] = pd.to_datetime(df['date']).dt.year
df['month'] = pd.to_datetime(df['date']).dt.month
df['day'] = pd.to_datetime(df['date']).dt.day
df['hour'] = pd.to_datetime(df['date']).dt.hour
df['minute'] = pd.to_datetime(df['date']).dt.minute
df['second'] = pd.to_datetime(df['date']).dt.second

df = df.drop(['date'], axis=1)

df

,company,rate,views,year,month,day,hour,minute,second
0,AMZN,129.330002,30,2023,6,26,4,0,0
1,GOOGL,122.339996,436,2023,6,26,4,0,0
2,MSFT,335.019989,137,2023,6,26,4,0,0
3,META,288.730011,682,2023,6,26,4,0,0
4,AAPL,186.679993,35,2023,6,26,4,0,0
5,AMZN,129.330002,10,2023,6,26,3,0,0
6,GOOGL,122.339996,429,2023,6,26,3,0,0
7,MSFT,335.019989,117,2023,6,26,3,0,0
8,META,288.730011,528,2023,6,26,3,0,0
9,AAPL,186.679993,49,2023,6,26,3,0,0


In [30]:
# predicting google rates
google_df = df[df.company == 'GOOGL']
x = google_df.drop(['rate' ,'company'], axis = 1)
y = google_df.loc[:, 'rate']

test_size = 0.2
number_of_test_rows = int(len(x) * test_size)

x_train = x.iloc[number_of_test_rows:,:].values
x_test = x.iloc[:number_of_test_rows, :].values

y_train = y[number_of_test_rows:].values
y_test = y[:number_of_test_rows].values

assert len(x_train) + len(x_test) == len(x)
assert len(y_train) + len(y_test) == len(y)


In [85]:
model = RandomForestRegressor(n_estimators=50)
model.fit(x_train, y_train)

RandomForestRegressor(n_estimators=50)

In [86]:
model.score(x_test, y_test)

-2.2818435836389765

In [87]:
y_predicted = model.predict(x_test)

In [88]:
y_predicted

array([121.03999634, 121.03999634])

In [89]:
x_test

array([[ 963, 2023,    6,   21,   15,    0,    0],
       [ 983, 2023,    6,   21,   14,    0,    0]])